In [2]:
import tensorflow as tf

In [9]:
from keras import Model
from keras.layers import Embedding, LSTM

Using TensorFlow backend.


In [ ]:
NUM_WORDS = 3000 #가장 많이 사용된 3000개

In [ ]:
class Encoder(Model):
    def __init__(self):
        super(Encoder, self).__init__() #Model(Encoder)?
        self.emb = Embedding(NUM_WORDS, 64)
        
        self.lstm = LSTM(512, return_state=True) #return_State : hidden, cell state 둘다 출력해주기 위한 옵션
        
    def __call__(self, x, training=False, mask=None):
        x = self.emb(x)
        _, h, c = self.lstm(x) #출력벡터인 hidden/cell state를 받음. we discard encoder outputs.
        return h, c

In [ ]:
class Decoder(Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = Embedding(NUM_WORDS, 64)
        
        self.lstm = LSTM(512, return_sequences=True, return_state=True) #h1,...hn / c1,...cn까지 모두 출력받음
        self.dense = Dense(NUM_WORDS, activation='softmax') #to make output
        
    def __call__(self, inputs, training=False, mask=None):
        #x for each inputs
        #h, c for inital state of h and c
        x, h, c = inputs
        x = self.emb(x) #x for what?
        
        outputs, _, _ = self.lstm(x, initial_state=[h,c]) 
        #x로 input 집어넣는다. 위로 나오는게 x(길이512). 옆으로 나오는게 h, c. discard h,c
        return self.dense(outputs) #각 output x 에 dense를 취해준다

In [12]:
class seq2seq(Model):
    def __init__(self, sos, eos):
        super(seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos
        
    def __call__(self, inputs, training=False, mask=None):
        if training: #y 값이 input으로 있음
            x, inp = inputs #x for encoder inputs. inp for decoder inputs
            
            h,c = self.enc(x)
            
            y = self.dec((inp,h,c)) #y는 softmax로 나타낸 모든 단어에 대한 확률값 (해당 위치에서의)
            
            return y #한꺼번에 y값 처리 가능
        
        else: #y값이 input으로 없음
            '''shifted output for inference'''
            x = inputs
            h,c = self.enc(x)
            
            #decoder단에 가장 먼저 sos삽입
            inp = tf.convert_to_tensor(self.sos)
            inp = tf.reshape(inp, (1,1)) #input길이가 하나밖에 없음
            
            seq = tf.TensorArray(tf.int32, 64) #정답array - 최대 64길이까지 출력으로 받는다(summarization을)
            
            #shifted output
            for _ in tf.range(64): #tf.range better for keras Models
                y = self.dec([inp,h,c]) #input길이 하나밖에 없기때문에 한개값 출력받을 것임
                
                #shifted output
                inp = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32) #for next input
                inp = tf.reshape(inp, (1,1))
                
                seq = seq.write(_,inp)
                
                if inp == self.eos:
                    break
            
            return tf.reshape(seq.stack(), (1,64)) # stack은 그동안 TensorArray로 받은 값을 쌓아주는 작업을 한다.   
            
            

In [ ]:
def train(model, inputs, labels, loss_object, opt, train_loss, train_acc):
    output_labels = labels[:,1:] #real labels - to compare with lstm outputs - 제일첫번째는 sos
    shifted_labels = labels[:,:-1] #shifted_labels & what goes for the decoder inputs - 제일마지막은 eos
    
    Input()